Reverse the z score normalisation of the nanopore pore models. HAs user tunable STD_DEV and MEAN values - which creates a current value for each kmer in the 9mer_levels_v1.txt file. Then creates squiggle from the reads found in testing_r10_fasta.fasta in the fasta directory, starts a guppy server instance and basecalles them, Then maps them to the chromsome 20 sequence in the fasta directory.

In [1]:
import collections
import re
import time as t
from contextlib import contextmanager, redirect_stdout
from datetime import datetime
from io import StringIO
from itertools import islice
from pathlib import Path
from typing import Union

import mappy as mp
import numpy as np
import numpy.typing as npt
import pandas as pd
from pyfastx import Fasta
from pyguppy_client_lib import helper_functions
from pyguppy_client_lib.pyclient import PyGuppyClient

In [18]:
MEAN = 100
STD_DEV = 24
BIN_PATH = "/usr/bin/"
FORMAT = "%d%m%y%H%M%S"
DIGITISATION = 1
RANGE = 1
PREFIX = "TTTTTTTTTTTTTTTTTTAATCAAGCAGCGGAGTTGAGGACGCGAGACGGGACTTTTTTAGCAGACTTTACGGACTACGACT"

In [19]:
time = datetime.now().strftime(FORMAT)
work_dir = Path("working") / time
work_dir.mkdir(exist_ok=True, parents=False)

In [20]:
df = pd.read_csv("9mer_levels_v1.txt", header=None, sep="\t", names=["kmer", "value"])

In [21]:
df["value"] = df["value"] * STD_DEV + MEAN

In [22]:
kmer_values = df.set_index("kmer").to_dict(orient="index")

In [23]:
df.to_csv(f"working/{time}/model_test.tsv", sep="\t", index=None, header=None)

In [24]:
def pack(read: dict[str : Union[str, npt.NDArray[np.int16]]]):
    """Pack an ont_fast5_api.Fast5Read for calling
    passed dict has two fields, read_id and raw_Data
    read_id: str, raw_data: npt.NDArray[np.int16]
    """
    read_id = read["read_id"]
    raw_data = read["raw_data"]
    scaling = RANGE / DIGITISATION
    offset = 0.0
    return helper_functions.package_read(read_id, raw_data, offset, scaling)

In [25]:
@contextmanager
def start_guppy_server_and_client(bin_path, config, port, server_args):
    server_args.extend(
        ["--config", config, "--port", port, "--log_path", str((Path(".") / "junk"))]
    )
    # This function has it's own prints that may want to be suppressed
    with redirect_stdout(StringIO()) as fh:
        server, port = helper_functions.run_server(server_args, bin_path=bin_path)

    if port == "ERROR":
        raise RuntimeError("Server couldn't be started")

    if port.startswith("ipc"):
        address = f"{port}"
    else:
        address = f"localhost:{port}"
    client = PyGuppyClient(address=address, config=config)

    try:
        with client:
            yield client
    finally:
        server.terminate()

In [26]:
def sliding_window(iterable, n):
    # sliding_window('ABCDEFG', 4) --> ABCD BCDE CDEF DEFG
    it = iter(iterable)
    window = collections.deque(islice(it, n), maxlen=n)
    if len(window) == n:
        yield tuple(window)
    for x in it:
        window.append(x)
        yield tuple(window)

In [27]:
def signalify(
    kmers: dict[str, dict[str, float]], sequence: str
) -> npt.NDArray[np.int16]:
    """convert a given sequence to signal using R10 models,
    returning np array containing 10 samples per base"""
    a = []
    # Always upper case signal
    for kmer in sliding_window(sequence.upper(), 9):
        value = kmers["".join(kmer)]["value"]
        value = (value * 2048) / 200 - 0
        for _ in range(10):
            a.append(value)
    return np.array(a).astype(np.int16)

In [28]:
prefix = signalify(kmer_values, PREFIX)

In [29]:
with start_guppy_server_and_client(
    BIN_PATH,
    "dna_r10.4.1_e8.2_400bps_fast.cfg",
    "ipc:///tmp/.guppy/5555",
    ["--device", "cuda:all"],
) as client:
    fa = Fasta("testing_r10_fasta.fasta")
    for seq in fa:
        # remove any non ACGT and replace with A
        sequence = re.sub(r"[^ACGT]", "A", seq.seq, flags=re.IGNORECASE)
        signal = signalify(kmer_values, sequence)
        signal = np.concatenate((prefix, signal))
        read = {"read_id": seq.name, "raw_data": signal}
        success_pass = client.pass_read(read, pack)
    t.sleep(1)
    res = client.get_completed_reads()
    aligner = mp.Aligner(
        "hg38_no_alts.part_NC_000020.11 Homo sapiens chromosome 20, GRCh38.p14 Primary Assembly.fa.gz"
    )
    for r in res:
        start, end = r[0]["metadata"]["read_id"].rsplit("_", 2)[-2:]
        read_number = r[0]["metadata"]["read_id"].rsplit("_", 3)[-3]
        sequence = r[0]["datasets"]["sequence"]
        print(len(sequence))
        als = aligner.map(sequence)
        for al in als:
            print(
                f"read_id: {r[0]['metadata']['read_id']} \n\treal start: {start}, real end : {end}\n\tmap_start {al.r_st}, map end {al.r_en}"
            )

9679
read_id: test_read_20_936710_946710 
	real start: 936710, real end : 946710
	map_start 936721, map end 946676
8818
read_id: test_read_18_511554_520554 
	real start: 511554, real end : 520554
	map_start 511567, map end 520479
8308
read_id: test_read_17_98418_106918 
	real start: 98418, real end : 106918
	map_start 98424, map end 106908
7713
read_id: test_read_16_220153_228153 
	real start: 220153, real end : 228153
	map_start 220166, map end 228091
7273
read_id: test_read_15_827036_834536 
	real start: 827036, real end : 834536
	map_start 827046, map end 834518
6809
read_id: test_read_14_398055_405055 
	real start: 398055, real end : 405055
	map_start 398067, map end 405048
6314
read_id: test_read_13_683244_689744 
	real start: 683244, real end : 689744
	map_start 683268, map end 689409
0
3009
read_id: test_read_6_66172_69172 
	real start: 66172, real end : 69172
	map_start 66335, map end 69164
546
read_id: test_read_1_140891_141391 
	real start: 140891, real end : 141391
	map_star